In [1]:
import pandas as pd
import numpy as np
import os
import re
from icecream import ic
from collections import defaultdict
import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
# import pickle

/tmp/ipykernel_1005649/2563711703.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# embeds_file = '/home/varunaja/mri_pet/ready_data/ADNI_ABTAU_MOREFEATURES.pkl'
# with open(embeds_file, 'rb') as file:
#     # Load the data from the pickle file
#     data = pickle.load(file)

In [3]:
def code_data_type(df, meta, percent_missing):
    for i, col in enumerate(df.columns):
        if col == 'ID':
            continue
        # print(col)
        # print(meta1[meta1['name'] == col]['type'].iloc[0])
        meta[col] = {}
        meta[col]['name'] = col
        meta[col]['unique_values'] = list(df[col].dropna().unique())
        meta[col]['num_unique_values'] = len(meta[col]['unique_values'])
        meta[col]['length'] = 1
        mydtype = df[col].dtype
        if 'category' in str(mydtype):
            meta[col]['type'] = 'C'
        else:
            meta[col]['type'] = 'N'
        if col == 'img_MRI_T1':
            meta[col]['type'] = 'M'
        meta[col]['missing_rate'] = round(percent_missing[col], 2)
    df = pd.DataFrame(meta).T[['name', 'missing_rate', 'type', 'num_unique_values', 'length', 'unique_values']]
    return df

# Load Data

In [4]:
# data = pd.read_csv('/home/varunaja/mri_pet/adrd_tool_varuna/adrd_transformer/data/adni_data_single.csv')
# data = pd.read_csv('/home/varunaja/mri_pet/adrd_tool_varuna/adrd_transformer/data/oasis_data_single.csv')
data = pd.read_csv('../../../adrd_tool_varuna/adrd_transformer/data/train_data.csv')
data.head()

,FS_MTL_VOLUME,FS_TEMPORAL_VOLUME,FS_PARIETAL_VOLUME,FS_OCCIPITAL_VOLUME,FS_FRONTAL_VOLUME,FS_ctx_lh_entorhinal_volume,FS_ctx_rh_entorhinal_volume,FS_left_hippocampus_volume,FS_right_hippocampus_volume,FS_left_amygdala_volume,FS_right_amygdala_volume,FS_ctx_lh_parahippocampal_volume,FS_ctx_rh_parahippocampal_volume,FS_ctx_lh_inferiortemporal_volume,FS_ctx_rh_inferiortemporal_volume,FS_ctx_lh_fusiform_volume,FS_ctx_rh_fusiform_volume,FS_ctx_lh_middletemporal_volume,FS_ctx_rh_middletemporal_volume,FS_ctx_lh_superiortemporal_volume,FS_ctx_rh_superiortemporal_volume,FS_ctx_lh_transversetemporal_volume,FS_ctx_rh_transversetemporal_volume,FS_ctx_lh_isthmuscingulate_volume,FS_ctx_rh_isthmuscingulate_volume,FS_ctx_lh_inferiorparietal_volume,FS_ctx_rh_inferiorparietal_volume,FS_ctx_lh_precuneus_volume,FS_ctx_rh_precuneus_volume,FS_ctx_lh_superiorparietal_volume,FS_ctx_rh_superiorparietal_volume,FS_ctx_lh_supramarginal_volume,FS_ctx_rh_supramarginal_volume,FS_ctx_lh_posteriorcingulate_volume,FS_ctx_rh_posteriorcingulate_volume,FS_ctx_lh_postcentral_volume,FS_ctx_rh_postcentral_volume,FS_ctx_lh_lateraloccipital_volume,FS_ctx_rh_lateraloccipital_volume,FS_ctx_lh_lingual_volume,FS_ctx_rh_lingual_volume,FS_ctx_lh_pericalcarine_volume,FS_ctx_rh_pericalcarine_volume,FS_ctx_lh_cuneus_volume,FS_ctx_rh_cuneus_volume,FS_ctx_lh_caudalanteriorcingulate_volume,FS_ctx_rh_caudalanteriorcingulate_volume,FS_ctx_lh_caudalmiddlefrontal_volume,FS_ctx_rh_caudalmiddlefrontal_volume,FS_ctx_lh_lateralorbitofrontal_volume,FS_ctx_rh_lateralorbitofrontal_volume,FS_ctx_lh_medialorbitofrontal_volume,FS_ctx_rh_medialorbitofrontal_volume,FS_ctx_lh_parsopercularis_volume,FS_ctx_rh_parsopercularis_volume,FS_ctx_lh_parsorbitalis_volume,FS_ctx_rh_parsorbitalis_volume,FS_ctx_lh_parstriangularis_volume,FS_ctx_rh_parstriangularis_volume,FS_ctx_lh_precentral_volume,FS_ctx_rh_precentral_volume,FS_ctx_lh_rostralanteriorcingulate_volume,FS_ctx_rh_rostralanteriorcingulate_volume,FS_ctx_lh_rostralmiddlefrontal_volume,FS_ctx_rh_rostralmiddlefrontal_volume,FS_ctx_lh_superiorfrontal_volume,FS_ctx_rh_superiorfrontal_volume,FS_ctx_lh_insula_volume,FS_ctx_rh_insula_volume,FS_wm_hypointensities_volume,ID,his_BIRTHMO,his_BIRTHYR,his_SEX,his_HISPANIC,his_PRIMLANG,his_EDUC,his_MARISTAT,his_LIVSIT,his_INDEPEND,his_RESIDENC,his_HANDED,his_NACCAGE,his_NACCNIHR,apoe_NACCNE4S,his_NACCFAM,his_NACCMOM,his_NACCDAD,med_ANYMEDS,med_NACCAMD,med_NACCAHTN,med_NACCHTNC,med_NACCACEI,med_NACCAAAS,med_NACCBETA,med_NACCCCBS,med_NACCDIUR,med_NACCVASD,med_NACCANGI,med_NACCLIPL,med_NACCNSD,med_NACCAC,med_NACCADEP,med_NACCAPSY,med_NACCAANX,med_NACCADMD,med_NACCPDMD,med_NACCEMD,med_NACCEPMD,med_NACCDBMD,his_TOBAC30,his_TOBAC100,his_SMOKYRS,his_PACKSPER,his_QUITSMOK,his_ALCOCCAS,his_ALCFREQ,his_CVHATT,his_CVAFIB,his_CVANGIO,his_CVBYPASS,his_CVPACDEF,his_CVCHF,his_CVANGINA,his_CVHVALVE,his_CBSTROKE,his_CBTIA,his_PD,his_SEIZURES,his_TBI,his_TBIBRIEF,his_TBIEXTEN,his_TBIWOLOS,his_TBIYEAR,his_DIABETES,his_DIABTYPE,his_HYPERTEN,his_HYPERCHO,his_B12DEF,his_THYROID,his_ARTHRIT,his_ARTHTYPE,his_ARTHUPEX,his_ARTHLOEX,his_ARTHSPIN,his_ARTHUNK,his_INCONTU,his_INCONTF,his_APNEA,his_RBD,his_INSOMN,his_ALCOHOL,his_PTSD,his_BIPOLAR,his_SCHIZ,his_DEP2YRS,his_ANXIETY,his_OCD,his_NPSYDEV,his_PSYCDIS,his_NACCTBI,ph_HEIGHT,ph_WEIGHT,ph_NACCBMI,ph_BPSYS,ph_BPDIAS,ph_HRATE,ph_VISION,ph_VISCORR,ph_VISWCORR,ph_HEARING,ph_HEARAID,ph_HEARWAID,npiq_DEL,npiq_HALL,npiq_AGIT,npiq_DEPD,npiq_ANX,npiq_ELAT,npiq_APA,npiq_DISN,npiq_IRR,npiq_MOT,npiq_NITE,npiq_APP,gds_SATIS,gds_DROPACT,gds_EMPTY,gds_BORED,gds_SPIRITS,gds_AFRAID,gds_HAPPY,gds_HELPLESS,gds_STAYHOME,gds_MEMPROB,gds_WONDRFUL,gds_WRTHLESS,gds_ENERGY,gds_HOPELESS,gds_BETTER,gds_NACCGDS,faq_BILLS,faq_TAXES,faq_SHOPPING,faq_GAMES,faq_STOVE,faq_MEALPREP,faq_EVENTS,faq_PAYATTN,faq_REMDATES,faq_TRAVEL,exam_PARKSIGN,exam_RESTTRL,exam_RESTTRR,exam_SLOWINGL,exam_SLOWINGR,exam_RIGIDL,exam_RIGIDR,exam_BRADY,exam_PARKGAIT,exam_POSTINST,exam_CVDSIGNS,exam_CORTDEF,exam_SIVDFIND,exam_CVDMOTL,exam_CVDM

In [5]:
data.drop(columns=['his_BIRTHMO', 'his_BIRTHYR'], inplace = True)

In [94]:
# data.drop(columns = ['cd_DIAGNOSIS'], inplace = True)

In [6]:
# demog = data.columns[data.columns.str.contains('his')].to_list()
demog_his = data.columns[data.columns.str.contains('his')].to_list()
# diagnosis = ['cd_DIAGNOSIS']
mmse = data.columns[data.columns.str.contains('bat') & data.columns.str.contains('MMSE')].to_list()
cdr = data.columns[data.columns.str.contains('cdr')].to_list()
cvd = data.columns[data.columns.str.contains('cvd')].to_list()
physical = data.columns[data.columns.str.contains('ph')].to_list()
moca = data.columns[data.columns.str.contains('bat') & data.columns.str.contains('MOCA')].to_list()
np = data.columns[(data.columns.str.contains('bat')) & (~data.columns.str.contains('MOCA')) & (~data.columns.str.contains('MMSE'))].to_list()
np_cat = ['bat_UDSBENRS', 'bat_COGSTAT', 'bat_CRAFTCUE']
np_num = [item for item in np if item not in np_cat]
gds = data.columns[data.columns.str.contains('gds')].to_list()
npiq = data.columns[data.columns.str.contains('npiq')].to_list()
faq = data.columns[data.columns.str.contains('faq')].to_list()
exam = data.columns[data.columns.str.contains('exam')].to_list()
meds = data.columns[data.columns.str.contains('med')].to_list()
dx = data.columns[data.columns.str.contains('cd_NACCUDSD')].to_list()
apoe = data.columns[data.columns.str.contains('apoe')].to_list()
# hx = data.columns[data.columns.str.contains('iha')].to_list()
# labels = ['amy_label', 'tau_label', 'NC', 'MCI', 'DE']
labels = ['amy_label', 'tau_label']
volumes = data.columns[data.columns.str.contains('FS')].to_list()

# All training data

In [7]:
all_columns = demog_his + mmse + volumes + cdr + cvd + physical + moca + np + gds + npiq + faq + exam + meds + dx + apoe + labels

numerical = ['his_EDUC'] + mmse + physical + volumes + cdr + np_num + ['his_NACCAGE', 'med_NACCAMD', 'his_SMOKYRS', 
                                                                   'his_PACKSPER', 'his_QUITSMOK', 'his_ALCFREQ', 
                                                                   'his_TBIYEAR', 'bat_MOCANAMI', 'bat_MOCASER7', 
                                                                   'bat_MOCARECN', 'bat_NACCMOCA', 'bat_TRAILA', 
                                                                   'bat_TRAILB', 'gds_NACCGDS'] 

# exclude from np 

# imaging = ['img_MRI_T1']

for col in all_columns:
    if col not in numerical: # and col not in imaging:
        data[col] = data[col].astype('category')

for col in numerical:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [8]:
meta = defaultdict()
percent_missing = dict(data.isnull().sum() / len(data))
filtered_percent_missing = {key: value for key, value in percent_missing.items() if value > 0.89}

filtered_percent_missing

{'his_QUITSMOK': 0.9321692411014103,
 'his_ALCFREQ': 0.9002686366689053,
 'his_TBIYEAR': 0.9778374748153122,
 'his_DIABTYPE': 0.9795164539959704,
 'his_ARTHTYPE': 0.9323931049921648,
 'ph_HEARWAID': 0.9430266398029997,
 'bat_PENTAGON': 0.9846653234833221,
 'bat_MEMTIME': 0.9857846429370942,
 'bat_DIGIF': 0.985113051264831,
 'bat_DIGIFLEN': 0.985113051264831,
 'bat_DIGIB': 0.985113051264831,
 'bat_DIGIBLEN': 0.985113051264831,
 'bat_BOSTON': 0.9242220729796284,
 'FDGAD': 0.9152675173494516,
 'SCANDATE.FDG': 0.9152675173494516,
 'NACCADC': 0.9716812178195657,
 'MRIT1': 0.9716812178195657,
 'MRIT2': 0.9716812178195657,
 'MRIFLAIR': 0.9716812178195657,
 'MRIFIELD': 0.9716812178195657,
 'MRIMANU': 0.9716812178195657,
 'MRIMODL': 0.9716812178195657,
 'NACCNMRI': 0.9716812178195657,
 'SCANDATE.MRI': 0.9716812178195657,
 'NACCMRFI': 0.9716812178195657}

In [9]:
df = data.drop(columns=filtered_percent_missing.keys())

In [10]:
for i, col in enumerate(data.columns):
    mydtype = data[col].dtype
    if 'category' in str(mydtype):
        # print(col)
        print(data[col].value_counts())

his_SEX
female    4917
male      4017
Name: count, dtype: int64
his_HISPANIC
no     7993
yes     455
Name: count, dtype: int64
his_PRIMLANG
0.0    7417
1.0     212
5.0      80
6.0      45
2.0      20
3.0       2
4.0       1
Name: count, dtype: int64
his_MARISTAT
0.0    6080
2.0    1065
1.0     835
4.0     387
5.0      60
3.0      31
Name: count, dtype: int64
his_LIVSIT
1.0    2538
0.0     723
3.0     221
2.0     213
4.0      34
Name: count, dtype: int64
his_INDEPEND
0.0    2656
1.0     674
2.0     257
3.0     104
Name: count, dtype: int64
his_RESIDENC
0.0    8268
1.0     157
3.0      39
2.0      38
Name: count, dtype: int64
his_HANDED
1.0    4401
0.0     469
2.0      45
Name: count, dtype: int64
his_NACCNIHR
whi    7541
blk     576
asi     231
mul     117
ind      26
haw       4
Name: count, dtype: int64
apoe_NACCNE4S
0.0    1537
1.0     964
2.0     235
Name: count, dtype: int64
his_NACCFAM
1.0    1647
0.0     974
Name: count, dtype: int64
his_NACCMOM
0.0    4227
1.0    3455
Name: coun

In [11]:
list(data.columns)

['FS_MTL_VOLUME',
 'FS_TEMPORAL_VOLUME',
 'FS_PARIETAL_VOLUME',
 'FS_OCCIPITAL_VOLUME',
 'FS_FRONTAL_VOLUME',
 'FS_ctx_lh_entorhinal_volume',
 'FS_ctx_rh_entorhinal_volume',
 'FS_left_hippocampus_volume',
 'FS_right_hippocampus_volume',
 'FS_left_amygdala_volume',
 'FS_right_amygdala_volume',
 'FS_ctx_lh_parahippocampal_volume',
 'FS_ctx_rh_parahippocampal_volume',
 'FS_ctx_lh_inferiortemporal_volume',
 'FS_ctx_rh_inferiortemporal_volume',
 'FS_ctx_lh_fusiform_volume',
 'FS_ctx_rh_fusiform_volume',
 'FS_ctx_lh_middletemporal_volume',
 'FS_ctx_rh_middletemporal_volume',
 'FS_ctx_lh_superiortemporal_volume',
 'FS_ctx_rh_superiortemporal_volume',
 'FS_ctx_lh_transversetemporal_volume',
 'FS_ctx_rh_transversetemporal_volume',
 'FS_ctx_lh_isthmuscingulate_volume',
 'FS_ctx_rh_isthmuscingulate_volume',
 'FS_ctx_lh_inferiorparietal_volume',
 'FS_ctx_rh_inferiorparietal_volume',
 'FS_ctx_lh_precuneus_volume',
 'FS_ctx_rh_precuneus_volume',
 'FS_ctx_lh_superiorparietal_volume',
 'FS_ctx_rh_supe

In [12]:
df = code_data_type(data, meta, percent_missing)

In [13]:
df.to_csv('../meta_files/train_meta_file_imaging_0422.csv', index=False)

# ADNI

In [88]:
demog = data.columns[data.columns.str.contains('his')].to_list()

# diagnosis = ['cd_DIAGNOSIS']
mmse = data.columns[data.columns.str.contains('bat') & data.columns.str.contains('MMSE')].to_list()
cdr = data.columns[data.columns.str.contains('cdr')].to_list()
cvd = data.columns[data.columns.str.contains('cvd')].to_list()
physical = data.columns[data.columns.str.contains('ph')].to_list()
moca = data.columns[data.columns.str.contains('bat') & data.columns.str.contains('MOCA')].to_list()
np = data.columns[(data.columns.str.contains('bat')) & (~data.columns.str.contains('MOCA')) & (~data.columns.str.contains('MMSE'))].to_list()
gds = data.columns[data.columns.str.contains('gds')].to_list()
npiq = data.columns[data.columns.str.contains('npiq')].to_list()
faq = data.columns[data.columns.str.contains('faq')].to_list()
exam = data.columns[data.columns.str.contains('exam')].to_list()
hx = data.columns[data.columns.str.contains('iha')].to_list()
labels = ['amy_label', 'tau_label', 'NC', 'MCI', 'DE']
labels = ['amy_label', 'tau_label']
volumes = data.columns[data.columns.str.contains('FS')].to_list()

In [ ]:
all_columns = demog + mmse + volumes + cdr + cvd + physical + moca + np + gds + npiq + faq + hx + labels

numerical = ['his_EDUC'] + mmse + physical + volumes + cdr + np + ['his_NACCAGE', 'bat_MOCANAMI', 'bat_MOCASER7', 'bat_MOCARECN', 'bat_NACCMOCA', 'bat_TRAILA', 'bat_TRAILB', 'gds_NACCGDS', 'npiq_NPISCORE', 'faq_FAQTOTAL', 'cvd_HACHIN'] 
# imaging = ['img_MRI_T1']

for col in all_columns:
    if col not in numerical: # and col not in imaging:
        data[col] = data[col].astype('category')

for col in numerical:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [ ]:
print(all_columns)

In [ ]:
meta = defaultdict()
percent_missing = dict(data.isnull().sum() / len(data))
percent_missing

In [ ]:
meta = defaultdict()
percent_missing = dict(data.isnull().sum() / len(data))
filtered_percent_missing = {key: value for key, value in percent_missing.items() if value > 0.89}
filtered_percent_missing

In [ ]:
for i, col in enumerate(data.columns):
    mydtype = data[col].dtype
    if 'category' in str(mydtype):
        # print(col)
        print(data[col].value_counts())

In [ ]:
df = code_data_type(data, meta, percent_missing)
df.to_csv('../meta_files/adni_meta_file.csv', index=False)

# OASIS

In [ ]:
# don't need this, should have removed sooner
data.drop(columns=['bat_CRAFTDTI', 'cvd_STROKCOG', 'cvd_CVDIMAG'], inplace = True)
np.remove('bat_CRAFTDTI')
cvd.remove('cvd_STROKCOG')
cvd.remove('cvd_CVDIMAG')

In [ ]:
all_columns = demog + mmse + volumes + cdr + np + cvd + physical + moca + gds + npiq + faq + hx + labels

numerical = ['his_EDUC'] + mmse + physical + volumes + cdr + np + ['his_NACCAGE', 'his_SMOKYRS', 'his_QUITSMOK', 'his_PACKSPER', 'bat_MOCANAMI', 'bat_MOCASER7', 'bat_MOCARECN', 'bat_NACCMOCA', 'gds_NACCGDS', 'faq_FAQTOTAL', 'cvd_HACHIN'] 
# imaging = ['img_MRI_T1']

for col in all_columns:
    if col not in numerical: # and col not in imaging:
        data[col] = data[col].astype('category')

for col in numerical:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [ ]:
meta = defaultdict()
percent_missing = dict(data.isnull().sum() / len(data))
filtered_percent_missing = {key: value for key, value in percent_missing.items() if value > 0.89}

filtered_percent_missing

In [ ]:
data['DE'].value_counts() 

In [ ]:
df = code_data_type(data, meta, percent_missing)
df

In [ ]:
print(all_columns)

In [ ]:
for i, col in enumerate(data.columns):
    mydtype = data[col].dtype
    if 'category' in str(mydtype):
        # print(col)
        print(data[col].value_counts())

In [ ]:
df.to_csv('../meta_files/oasis_meta_file.csv', index=False)